# CyLinter Pipeline - GUI-basierter Workflow

**Dieses Notebook steuert CyLinter QC ausschlie?lich ?ber GUIs. Konfiguration (Zelle 8) ? Ausf?hrung (Zelle 9); Codezellen bleiben unver?ndert.**

---

## ?? Notebook-?berblick
| # | Typ | Zweck | Pflicht? |
|---|-----|-------|----------|
|1|MD|?bersicht & Workflow-Hinweise|??|
|2|Code|Setup & Pfade (BASE_DIR, CONFIG_PATH, markers.csv)|? Run All|
|3|Code|`cylinter_config.yml` + `markers.csv` laden/validieren|? Run All|
|4|MD|Kurze Erkl?rung zur Config|??|
|5|Code|Manuelle Quantifizierung & CSV-Erstellung|?? Optional|
|6|Code|Pre-Flight Check (Pfad- & Schema-Validierung)|? Run All|
|7|MD|Intro zur Konfigurations-GUI|??|
|8|Code|**?? Konfig-GUI** (Marker-Subset w?hlen & speichern)|?? Tag 1|
|9|Code|**?? PIPELINE-START** (Checkpoint-aware)|?? Tag 1|
|10|Code|**STOP** ? Tag?1 endet hier|??|
|11|MD|Intro zur erweiterten Steuerung|??|
|12|Code|**?? Erweiterte GUI** (Tag?2+ / Fehlerkorrektur)|?? Tag 2+|
|13|MD|Hinweis zum Parquet-Viewer|?? Optional|
|14|Code|Parquet Viewer (Checkpoint-Dateien inspizieren)|?? Optional|

---

## ?? WORKFLOW TAG 1 (Erste Analyse)

### 1?? Notebook starten
```
Kernel ? Restart & Run All
```
- ? F?hrt Zellen 2?6 aus (Setup, Config-Load, optionale manuelle Quantifizierung, Pre-Flight-Check).
- ? L?dt Konfigurations-GUI (Zelle 8) und Pipeline-Start (Zelle 9).
- ? **Stoppt an der STOP-Zelle** (verhindert versehentlichen Tag-2-Start).

### 2?? Marker ausw?hlen (Zelle 8)
**Quick-Optionen:** Alle Marker, Alle au?er AF, Nur Bio (Counts basieren auf `markers.csv`).  
**Custom-Option:** Bio-Marker per Checkbox (AF wird automatisch ausgeschlossen).  
**Button ??? Speichern & Bereit?:** Schreibt `markersToExclude` in `cylinter_config.yml`, startet keine Pipeline.

### 3?? Pipeline starten (Zelle 9)
- Zeigt Marker-Zusammenfassung und 15 Module.
- **Button ??? Pipeline starten?** startet beim ersten fehlenden Checkpoint und l?uft bis `curateThumbnails`.
- Interactive Module (`selectROIs`, `setContrast`, `gating`) ?ffnen GUIs.

**?? Tag?1 Tipp:** Pipeline nicht abbrechen; Checkpoints werden nur beim vollst?ndigen Lauf geschrieben.

---

## ?? WORKFLOW TAG 2+ (Neue Marker oder Fehlerkorrektur)

1?? Notebook neu starten, nur Zellen 2?3 ausf?hren (Setup + Config).  
2?? Zelle 12 ?ffnen ? Erweiterte GUI.  
3?? Tab 1: Marker erg?nzen/mergen (Backup wird erstellt).  
4?? Tab 2: Checkpoints gezielt l?schen, Startmodul w?hlen, ?Run Selected?.  

Gating nutzt vorhandene Thresholds aus `cylinter_report.yml`; nur Marker ohne Threshold werden neu gegated.

---

## ?? WORKFLOW FEHLERKORREKTUR (einzelnes Modul neu starten)

- Notebook neu starten ? Zellen 2?3 ausf?hren.  
- Zelle 12 (Tab 2): fehlerhaftes Modul resetten, Startmodul w?hlen ? ?Run Selected?.  
- Pipeline l?uft bis Ende, ?berspringt bestehende Checkpoints.

---

## ?? Eingaben & Modul-Liste
- Erwartet `cylinter_config.yml` und `markers.csv` im Notebook-Ordner; Checkpoints/Reports: `cylinter_output_prune_test/`.
- Pipeline-Module (15):
```
1. aggregateData
2. selectROIs (GUI)
3. intensityFilter
4. areaFilter
5. cycleCorrelation
6. logTransform
7. pruneOutliers
8. metaQC
9. PCA
10. setContrast (GUI)
11. gating (GUI)
12. clustering
13. clustermap
14. frequencyStats
15. curateThumbnails
```

**Interactive Module (?ffnen GUIs):**
- `selectROIs`, `setContrast`, `gating`

---

##  CHECKPOINT-SYSTEM VERSTEHEN

**Was sind Checkpoints?**
- Nach jedem erfolgreichen Modul speichert CyLinter eine `.parquet` Datei
- Speicherort: `cylinter_output_prune_test/checkpoints/`
- Format: `module_name.parquet` (z.B. `aggregateData.parquet`, `gating.parquet`)

**Wie funktioniert es?**
- Beim Pipeline-Start pr?ft CyLinter, welche Checkpoints existieren
- Module mit Checkpoints werden **automatisch ?bersprungen**
- Nur Module ohne Checkpoint oder mit gel?schtem Checkpoint laufen neu

**Warum ist das n?tzlich?**
- ? **Zeit sparen:** Keine unn?tigen Neuberechnungen
- ? **Incremental Work:** Neue Marker hinzuf?gen ohne alles neu zu machen
- ? **Fehlerkorrektur:** Einzelne Module isoliert neu starten
- ? **Experimente:** Verschiedene Parameter testen (z.B. Clustering-Params)

**Gating-Thresholds:**
- Werden in `cylinter_report.yml` gespeichert (nicht in Checkpoint!)
- Persistieren ?ber Pipeline-L?ufe hinweg
- Erm?glichen TAG 2 Workflow: Neue Marker ohne alte Marker neu zu gaten

---

## ?? WICHTIGE HINWEISE

### CyLinter's Pipeline-Verhalten:
- **`--module X` setzt nur STARTPUNKT, nicht Endpunkt!**
- Pipeline l?uft IMMER vom Startmodul bis zum Ende (15. Modul)
- Sie k?nnen KEINE Modul-Teilmenge ausw?hlen (z.B. nur Module 3-5)
- Checkpoints erm?glichen das ?berspringen: Pipeline l?uft alle 15 durch, aber existierende Checkpoints werden ?bersprungen

### TAG 1 vs TAG 2:
- **TAG 1:** Vollst?ndige Erstanalyse
  - Zelle 8: Marker w?hlen
  - Zelle 9: Pipeline starten (aggregateData ? Ende)
  - Alle interactive Module durchlaufen
  
- **TAG 2+:** Inkrementelle Updates
  - Zelle 12 Tab 1: Neue Marker mergen
  - Zelle 12 Tab 2: Checkpoint l?schen + Pipeline starten
  - Checkpoints erm?glichen ?berspringen bereits berechneter Schritte

### Laufzeiten (ca.):
- **TAG 1 (AF ausgeschlossen):** ~40-50 Min
- **TAG 1 (alle Marker):** ~90-120 Min
- **TAG 2 (neue Marker):** ~10-15 Min
- **Fehlerkorrektur (1 Modul):** ~2-10 Min (je nach Modul)


In [ ]:
# Zelle 1: Imports und Basiskonfiguration (DYNAMISCH fÃ¼r beliebige Samples)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import yaml
import time
import os
import subprocess # FÃ¼r CLI-Aufrufe
import shutil

print("--- Zelle 1: Modulimporte, Pfaddefinitionen und automatische Dateierkennung ---")
BASE_DIR = Path.cwd()
CONFIG_PATH = BASE_DIR / "cylinter_config.yml"
MARKERS_CSV_PATH = BASE_DIR / "markers.csv"

print(f"Projekt-Basisverzeichnis (BASE_DIR): {BASE_DIR}")
print(f"Konfigurationsdatei (CONFIG_PATH): {CONFIG_PATH}")
print(f"Marker CSV (MARKERS_CSV_PATH): {MARKERS_CSV_PATH}")

if not CONFIG_PATH.exists():
    raise FileNotFoundError(f"cylinter_config.yml nicht gefunden: {CONFIG_PATH}")
if not MARKERS_CSV_PATH.exists():
    raise FileNotFoundError(f"markers.csv nicht gefunden: {MARKERS_CSV_PATH}")

# Sicherstellen, dass die Eingabeordner existieren
(BASE_DIR / 'csv').mkdir(exist_ok=True)
(BASE_DIR / 'tif').mkdir(exist_ok=True)
(BASE_DIR / 'seg').mkdir(exist_ok=True)
(BASE_DIR / 'mask').mkdir(exist_ok=True)

print("\n--- ÃœberprÃ¼fe vorhandene Dateien ---")
# ZÃ¤hle Dateien in jedem Ordner
for folder in ['csv', 'tif', 'seg', 'mask']:
    folder_path = BASE_DIR / folder
    if folder_path.exists():
        files = list(folder_path.glob('*'))
        files = [f for f in files if f.is_file()]  # Nur Dateien, keine Ordner
        print(f"{folder.upper()}: {len(files)} Datei(en) gefunden")
        if files:
            for f in files[:3]:  # Zeige max. 3 Dateien
                print(f"  - {f.name}")
            if len(files) > 3:
                print(f"  ... und {len(files)-3} weitere")

# --- OPTIONALE DATEIVORBEREITUNG ---
# Falls .tiff Dateien vorhanden sind, diese zu .tif konvertieren
print("\n--- PrÃ¼fe auf .tiff Dateien (falls vorhanden) ---")
tiff_converted = False
for folder in ['tif', 'seg', 'mask']:
    folder_path = BASE_DIR / folder
    if folder_path.exists():
        tiff_files = list(folder_path.glob("*.tiff"))
        if tiff_files:
            print(f"{folder.upper()}: Konvertiere {len(tiff_files)} .tiff Datei(en)...")
            for tiff_file in tiff_files:
                tif_file = tiff_file.with_suffix('.tif')
                shutil.move(tiff_file, tif_file)
                print(f"   â†’ Konvertiert: {tiff_file.name} â†’ {tif_file.name}")
                tiff_converted = True
        else:
            print(f"{folder.upper()}: Keine .tiff Dateien gefunden (OK)")

if not tiff_converted:
    print("\nKeine .tiff Konvertierung nÃ¶tig - alle Dateien bereits im .tif Format.")

print("\n--- Dateivorbereitung abgeschlossen ---")

# --- WICHTIG: markers.csv NICHT modifizieren (muss OME-TIFF Namen entsprechen) ---
print("\n--- PrÃ¼fe markers.csv Format ---")
# KRITISCH: markers.csv muss die ORIGINALEN Kanalnamen aus OME-TIFF enthalten (OHNE _cX)!
# Die _cX Suffixe gehÃ¶ren NUR in die CSV-Datei (Quantifizierung), NICHT in markers.csv

import pandas as pd
markers_original_path = BASE_DIR / "markers.csv"
markers_backup_path = BASE_DIR / "markers_original_backup.csv"

# Backup erstellen (nur einmal)
if not markers_backup_path.exists():
    import shutil
    shutil.copy(markers_original_path, markers_backup_path)
    print(f"  Original markers.csv gesichert als: {markers_backup_path.name}")

# Markers.csv laden und prÃ¼fen
df_markers = pd.read_csv(markers_original_path)
print(f"  Geladene markers.csv: {len(df_markers)} EintrÃ¤ge")

# CHECK & AUTO-FIX: PrÃ¼fe ob _cX vorhanden ist und fÃ¼ge hinzu falls nÃ¶tig
sample_marker_names = df_markers['marker_name'].head(3).tolist()
has_cycle_suffix = any('_c' in str(m) for m in sample_marker_names)

if not has_cycle_suffix:
    print(f"  âš ï¸  markers.csv hat KEINE _cX Suffixe: {sample_marker_names}")
    print(f"  â†’ FÃ¼ge _cX automatisch hinzu (fÃ¼r aggregateData)...")
    
    # FÃ¼ge _cX Suffixe hinzu (z.B. "DAPI" â†’ "DAPI_c1")
    df_markers['marker_name'] = (
        df_markers['marker_name'].str.strip().str.replace(' ', '_', regex=False) + 
        '_c' + df_markers['cycle_number'].astype(str)
    )
    
    # Speichern
    df_markers.to_csv(markers_original_path, index=False)
    print(f"  âœ… markers.csv korrigiert: _cX Suffixe hinzugefÃ¼gt")
    print(f"     Beispiele: {df_markers['marker_name'].head(3).tolist()}")
else:
    print(f"  âœ… markers.csv hat _cX Suffixe (korrekt fÃ¼r aggregateData): {sample_marker_names}")
    print(f"  â†’ CyLinter aggregateData kann CSV-Spalten finden!")

# Leerzeichen entfernen (falls vorhanden)
if df_markers['marker_name'].str.contains(' ', regex=False).any():
    print(f"  âš™ï¸  Entferne Leerzeichen aus Marker-Namen...")
    df_markers['marker_name'] = df_markers['marker_name'].str.strip().str.replace(' ', '_', regex=False)
    df_markers.to_csv(markers_original_path, index=False)
    print(f"  âœ… Leerzeichen entfernt")

print("\n--- PrÃ¼fung abgeschlossen ---")
print("Zelle 1 erfolgreich ausgefÃ¼hrt.\n")

In [ ]:
# Zelle 2: Konfigurationsobjekt erstellen (vereinfacht fÃ¼r Klarheit, da CLI die Config liest)
print("--- Zelle 2: Lade und validiere Konfigurationsdaten aus YAML ---")

if not CONFIG_PATH.exists():
    raise FileNotFoundError(f"FEHLER: {CONFIG_PATH} nicht gefunden. Zelle 1 erneut ausfÃ¼hren oder Pfad prÃ¼fen.")

try:
    with open(CONFIG_PATH, 'r') as f_yaml:
        config_dict_from_yaml = yaml.safe_load(f_yaml) # Diese Variable wird in Zelle 2.5 verwendet
    if config_dict_from_yaml is None:
        raise ValueError("cylinter_config.yml konnte nicht geladen werden oder ist leer.")
    print(f"cylinter_config.yml erfolgreich als Dictionary geladen.")
except Exception as e:
    print(f"FEHLER beim Laden der cylinter_config.yml: {e}")
    raise

try:
    markers_df_for_quant = pd.read_csv(MARKERS_CSV_PATH)
    if 'marker_name' not in markers_df_for_quant.columns:
        raise ValueError("Spalte 'marker_name' nicht in markers.csv gefunden.")
    if 'cycle_number' not in markers_df_for_quant.columns:
        raise ValueError("Spalte 'cycle_number' nicht in markers.csv gefunden.")
    
    # WICHTIG: markers.csv enthÃ¤lt bereits _cX Suffixe (nach Auto-Fix in Zelle 1)
    # Verwende diese DIREKT ohne weitere Modifikation
    all_channel_names_from_markers_csv = (
        markers_df_for_quant['marker_name'].str.strip().str.replace(' ', '_', regex=False)
    ).tolist()
    
    print(f"Marker-Informationen aus {MARKERS_CSV_PATH} geladen. {len(all_channel_names_from_markers_csv)} KanÃ¤le gefunden.")
    print(f"  Beispiele (bereits mit _cX aus markers.csv): {all_channel_names_from_markers_csv[:5]}")
except Exception as e:
    print(f"FEHLER beim Laden der markers.csv: {e}")
    raise

try:
    inDir_name_from_yaml = config_dict_from_yaml.get('inDir', '.')
    inDir_for_quant = (BASE_DIR / Path(inDir_name_from_yaml)).resolve()
    image_dir_name_from_yaml = config_dict_from_yaml.get('image_dir', 'tif')
    mask_dir_name_from_yaml = config_dict_from_yaml.get('mask_dir', 'mask')
    csv_dir_name_from_yaml = config_dict_from_yaml.get('csv_dir', 'csv')
    smd = config_dict_from_yaml.get('sampleMetadata', {})
    if not smd: raise ValueError("'sampleMetadata' fehlt in Config.")
    sample_key_from_yaml = list(smd.keys())[0]
    sample_values_from_yaml = smd[sample_key_from_yaml]
    SAMPLE_NAME_FOR_FILES_QUANT = sample_values_from_yaml[0]
    print(f"FÃ¼r Quantifizierung wird Sample-Dateiname verwendet: {SAMPLE_NAME_FOR_FILES_QUANT}")
    path_image_for_quant = inDir_for_quant / image_dir_name_from_yaml / f"{SAMPLE_NAME_FOR_FILES_QUANT}.ome.tif"
    path_cell_mask_for_quant = inDir_for_quant / mask_dir_name_from_yaml / f"{SAMPLE_NAME_FOR_FILES_QUANT}.tif"
    path_csv_output_for_quant = inDir_for_quant / csv_dir_name_from_yaml / f"{SAMPLE_NAME_FOR_FILES_QUANT}.csv"
    print(f"  Erwarteter Pfad fÃ¼r Bild: {path_image_for_quant}")
    print(f"  Erwarteter Pfad fÃ¼r Zellmaske: {path_cell_mask_for_quant}")
    print(f"  Erwarteter Pfad fÃ¼r Ausgabe-CSV: {path_csv_output_for_quant}")
except KeyError as ke: print(f"FEHLER: SchlÃ¼ssel in config nicht gefunden: {ke}"); raise
except Exception as e: print(f"FEHLER beim Extrahieren von Werten aus Config: {e}"); raise
print("Zelle 2 erfolgreich ausgefÃ¼hrt.\n")

---

## ðŸ“‹ Zelle 2: Konfiguration laden

LÃ¤dt `cylinter_config.yml` und `markers.csv`, validiert Struktur und bereitet Pfade vor.

In [ ]:
# Zelle 2.5: Manuelle Quantifizierung und Erstellung der CSV-Datei (DYNAMISCH)
print("--- Zelle 2.5: Manuelle Quantifizierung und Erstellung der CSV-Datei ---")

from skimage.measure import regionprops_table, regionprops
import tifffile
import pandas as pd
import numpy as np
from pathlib import Path

# ÃœberprÃ¼fe, ob die notwendigen Variablen aus Zelle 2 existieren
if 'config_dict_from_yaml' not in locals():
    raise NameError("config_dict_from_yaml ist nicht definiert. FÃ¼hren Sie Zelle 2 erneut aus.")
if 'all_channel_names_from_markers_csv' not in locals():
    raise NameError("all_channel_names_from_markers_csv ist nicht definiert. FÃ¼hren Sie Zelle 2 erneut aus.")
if 'SAMPLE_NAME_FOR_FILES_QUANT' not in locals():
    raise NameError("SAMPLE_NAME_FOR_FILES_QUANT ist nicht definiert. FÃ¼hren Sie Zelle 2 erneut aus.")

# --- DYNAMISCHE DATEIERKENNUNG (wie in Zelle 1) ---
def find_first_file(directory, extensions, exclude_patterns=None):
    """Findet die erste Datei mit passender Endung"""
    if exclude_patterns is None:
        exclude_patterns = []
    dir_path = BASE_DIR / directory
    if not dir_path.exists():
        return None
    for ext in extensions:
        for file in sorted(dir_path.glob(f"*{ext}")):
            if any(pattern in file.name for pattern in exclude_patterns):
                continue
            return file
    return None

print("\n--- Suche vorhandene Dateien fÃ¼r Quantifizierung ---")

# TIF: Suche tatsÃ¤chlich vorhandene Datei
tif_file = find_first_file('tif', ['.ome.tif', '.tif'], exclude_patterns=['.tiff'])
if not tif_file:
    raise FileNotFoundError(f"FEHLER: Keine TIF-Datei in {BASE_DIR / 'tif'} gefunden!")
path_image_for_quant = tif_file
print(f"Verwende TIF: {path_image_for_quant.name}")

# MASK: Suche tatsÃ¤chlich vorhandene Datei
mask_file = find_first_file('mask', ['.tif', '.tiff'], exclude_patterns=[])
if not mask_file:
    raise FileNotFoundError(f"FEHLER: Keine Mask-Datei in {BASE_DIR / 'mask'} gefunden!")
path_cell_mask_for_quant = mask_file
print(f"Verwende MASK: {path_cell_mask_for_quant.name}")

# CSV: Ziel fÃ¼r Ausgabe
path_csv_output_for_quant = BASE_DIR / 'csv' / f"{SAMPLE_NAME_FOR_FILES_QUANT}.csv"
print(f"Ausgabe CSV: {path_csv_output_for_quant.name}")
print("--- Dateien gefunden ---\n")

markers_to_exclude_for_quant = config_dict_from_yaml.get('markersToExclude', [])
quantification_channel_names = [
    name for name in all_channel_names_from_markers_csv
    if name not in markers_to_exclude_for_quant
]
print(f"FÃ¼r die Quantifizierung werden {len(quantification_channel_names)} KanÃ¤le verwendet (nach Ausschluss).")
print(f"  Ausgeschlossene Marker fÃ¼r Quant (aus config): {markers_to_exclude_for_quant[:5]}... (bis zu 5 gezeigt)")
print(f"  Zu quantifizierende Marker: {quantification_channel_names[:5]}... (bis zu 5 gezeigt)")

# --- Bild und Maske laden ---
print(f"Lade Bild fÃ¼r Quantifizierung: {path_image_for_quant}")
if not path_image_for_quant.exists():
    raise FileNotFoundError(f"Bilddatei nicht gefunden: {path_image_for_quant}")
full_image_data = tifffile.imread(path_image_for_quant)
print(f"  Bildform roh: {full_image_data.shape}")

# Bilddaten in (H, W, C) bringen, falls nÃ¶tig (Logik von Ihnen Ã¼bernommen)
if full_image_data.ndim == 3 and full_image_data.shape[0] == len(all_channel_names_from_markers_csv):
    full_image_data_hwc = np.transpose(full_image_data, (1, 2, 0))
elif full_image_data.ndim == 3 and full_image_data.shape[2] == len(all_channel_names_from_markers_csv):
    full_image_data_hwc = full_image_data
elif full_image_data.ndim == 2 and len(all_channel_names_from_markers_csv) == 1:
    full_image_data_hwc = np.expand_dims(full_image_data, axis=-1)
else:
    raise ValueError(f"Unerwartete Bilddimensionen {full_image_data.shape} fÃ¼r {len(all_channel_names_from_markers_csv)} KanÃ¤le.")
print(f"  Bildform fÃ¼r Quantifizierung (H,W,C): {full_image_data_hwc.shape}")

print(f"Lade Zellmaske fÃ¼r Quantifizierung: {path_cell_mask_for_quant}")
if not path_cell_mask_for_quant.exists():
    raise FileNotFoundError(f"Zellmaskendatei nicht gefunden: {path_cell_mask_for_quant}")
cell_mask = tifffile.imread(path_cell_mask_for_quant) # WICHTIG: cell_mask wird hier definiert
print(f"  Zellmaskenform: {cell_mask.shape}, Max Label: {np.max(cell_mask)}")

# WICHTIG: Validiere dass Image und Maske kompatible Shapes haben
if cell_mask.shape[:2] != full_image_data_hwc.shape[:2]:
    print(f"\nâš ï¸  WARNUNG: Shape-Mismatch!")
    print(f"   Image (H,W,C): {full_image_data_hwc.shape}")
    print(f"   Mask  (H,W):   {cell_mask.shape}")
    print(f"\n   â†’ Passe Maske an Image-Dimensionen an...")
    
    from skimage.transform import resize
    # Resize Maske auf Image-Dimensionen (nearest neighbor fÃ¼r Labels!)
    cell_mask_resized = resize(
        cell_mask.astype(float), 
        full_image_data_hwc.shape[:2], 
        order=0,  # Nearest neighbor (wichtig fÃ¼r Labels!)
        preserve_range=True,
        anti_aliasing=False
    ).astype(cell_mask.dtype)
    cell_mask = cell_mask_resized
    print(f"   âœ“ Maske angepasst auf: {cell_mask.shape}")

# --- DataFrame Erstellung und Quantifizierung ---
if np.max(cell_mask) == 0:
    print("WARNUNG: Zellmaske enthÃ¤lt keine Labels (max Label = 0). Erstelle leere CSV.")
    df_features = pd.DataFrame() # df_features wird hier fÃ¼r den leeren Fall definiert
else:
    print("Berechne morphologische Eigenschaften...")
    features_table = regionprops_table(
        cell_mask,
        properties=('label', 'area', 'centroid', 'major_axis_length',
                    'minor_axis_length', 'eccentricity', 'solidity', 'orientation')
    )
    df_features = pd.DataFrame(features_table) # df_features wird hier fÃ¼r den nicht-leeren Fall definiert
    print(f"  Spalten direkt nach regionprops_table: {df_features.columns.tolist()}")

    df_features.rename(columns={
        'label': 'CellID', 'area': 'Area', 'centroid-0': 'Y_centroid', 'centroid-1': 'X_centroid',
        'major_axis_length': 'MajorAxisLength', 'minor_axis_length': 'MinorAxisLength',
        'eccentricity': 'Eccentricity', 'solidity': 'Solidity', 'orientation': 'Orientation'
    }, inplace=True)
    print(f"  Spalten nach Umbenennung: {df_features.columns.tolist()}")

    if 'Area' in df_features.columns:
        df_features['Area'] = df_features['Area'].astype(float)
    else:
        print("WARNUNG: Spalte 'Area' nach Umbenennung immer noch nicht gefunden!")

    print(f"\nExtrahiere IntensitÃ¤ten fÃ¼r {len(quantification_channel_names)} ausgewÃ¤hlte Marker pro Zelle...")
    print("  Strategie: Mean (Standard) + 95th Percentile (fÃ¼r Membranmarker)")
    original_channel_name_to_index = {name: i for i, name in enumerate(all_channel_names_from_markers_csv)}
    
    # Lade markers.csv fÃ¼r localization Info
    markers_df_loc = pd.read_csv(MARKERS_CSV_PATH)
    marker_localization = dict(zip(markers_df_loc['marker_name'], markers_df_loc.get('localization', ['unknown']*len(markers_df_loc))))
    
    # PrÃ¼fe ob quantificationMetrics in config vorhanden
    compute_p95 = config_dict_from_yaml.get('quantificationMetrics', {}).get('compute_percentile_95', False)
    
    # Progress tracking
    from tqdm.auto import tqdm
    membrane_count = 0
    progress_bar = tqdm(quantification_channel_names, desc="Quantifiziere Marker", unit="marker")
    
    for marker_name_to_quantify in progress_bar:
        if marker_name_to_quantify not in original_channel_name_to_index:
            print(f"    WARNUNG: Zu quantifizierender Marker '{marker_name_to_quantify}' nicht in ursprÃ¼nglichen Kanalnamen gefunden. Ãœberspringe.")
            if not df_features.empty: df_features[marker_name_to_quantify] = np.nan # Nur hinzufÃ¼gen, wenn df nicht leer
            continue

        channel_index = original_channel_name_to_index[marker_name_to_quantify]
        is_membrane = marker_localization.get(marker_name_to_quantify, 'unknown') == 'membrane'

        if channel_index < full_image_data_hwc.shape[2]:
            current_channel_image = full_image_data_hwc[:, :, channel_index]
            
            # Berechne MEAN (Standard fÃ¼r alle Marker)
            intensity_props = regionprops_table(label_image=cell_mask, intensity_image=current_channel_image,
                                                properties=['label', 'intensity_mean'])
            df_intensity_marker = pd.DataFrame(intensity_props)
            df_intensity_marker.rename(columns={'label': 'CellID', 'intensity_mean': marker_name_to_quantify}, inplace=True)
            
            # Berechne 95th PERCENTILE fÃ¼r Membranmarker (wenn aktiviert)
            if compute_p95 and is_membrane:
                membrane_count += 1
                progress_bar.set_postfix({"Marker": marker_name_to_quantify[:20], "Type": "membrane+p95"})
                
                # OPTIMIERTE METHODE: ~5-10x schneller als Loop
                from scipy.ndimage import labeled_comprehension
                cell_ids = df_intensity_marker['CellID'].values
                p95_values = labeled_comprehension(
                    current_channel_image,
                    cell_mask,
                    cell_ids,
                    lambda pixels: np.percentile(pixels, 95) if len(pixels) > 0 else np.nan,
                    float,
                    np.nan
                )
                df_intensity_marker[f'{marker_name_to_quantify}_p95'] = p95_values
            else:
                progress_bar.set_postfix({"Marker": marker_name_to_quantify[:20], "Type": "mean-only"})
            
            # Merge mit Haupttabelle
            if df_features.empty and not df_intensity_marker.empty: # Wenn df_features leer war, aber jetzt IntensitÃ¤ten da sind
                df_features = df_intensity_marker
            elif not df_features.empty and not df_intensity_marker.empty :
                 df_features = pd.merge(df_features, df_intensity_marker, on='CellID', how='left')
            elif not df_intensity_marker.empty : # Sollte nicht passieren, wenn df_features leer und Maske leer war
                 print(f"WARNUNG: df_features ist leer, aber IntensitÃ¤tsdaten fÃ¼r {marker_name_to_quantify} vorhanden. Merging nicht mÃ¶glich.")

        else:
            print(f"    WARNUNG: Kanalindex {channel_index} fÃ¼r Marker '{marker_name_to_quantify}' ist auÃŸerhalb der Bilddimensionen "
                  f"(max Index ist {full_image_data_hwc.shape[2]-1}). Ãœberspringe IntensitÃ¤tsmessung.")
            if not df_features.empty: df_features[marker_name_to_quantify] = np.nan
    
    if compute_p95:
        print(f"  âœ“ {membrane_count} Membranmarker bekamen zusÃ¤tzlich _p95 Spalten")

# --- WICHTIG: Stabilisiere ALLE IntensitÃ¤tswerte (Cylinter macht log-Transformationen) ---
print("\n--- Stabilisiere IntensitÃ¤tswerte (ersetze â‰¤0 mit Minimum) ---")
intensity_cols = [
    col for col in df_features.columns
    if isinstance(col, str) and ('_c' in col or 'intensity' in col.lower())
]

stabilized_count = 0
for col in intensity_cols:
    series = df_features[col].astype(float)
    
    # Finde Minimalwert aller positiven Werte
    positive = series[series > 0]
    if not positive.empty:
        floor = positive.min()
    else:
        floor = 1.0
    
    # Ersetze alle Null-/Negativwerte
    replaced_mask = series <= 0
    if replaced_mask.any():
        df_features.loc[replaced_mask, col] = floor
        stabilized_count += 1
        print(f"  âœ“ {col}: {replaced_mask.sum()} Werte ersetzt durch {floor:.3g}")
    
    # ZusÃ¤tzlich: Ersetze NaN/inf Werte
    invalid_mask = ~np.isfinite(df_features[col])
    if invalid_mask.any():
        df_features.loc[invalid_mask, col] = floor
        print(f"  âœ“ {col}: {invalid_mask.sum()} NaN/inf Werte ersetzt")

if stabilized_count == 0:
    print("  âœ“ Alle IntensitÃ¤tswerte bereits positiv (keine Stabilisierung nÃ¶tig)")
else:
    print(f"  âœ“ {stabilized_count} Spalten stabilisiert")
print("--- Stabilisierung abgeschlossen ---\n")

# --- Sicherstellen, dass der Zielordner fÃ¼r die CSV existiert ---
if 'path_csv_output_for_quant' not in locals(): # Sollte durch Zelle 2 definiert sein
     raise NameError("Variable 'path_csv_output_for_quant' ist nicht definiert.")

csv_output_directory = path_csv_output_for_quant.parent
try:
    csv_output_directory.mkdir(parents=True, exist_ok=True)
    print(f"INFO: Zielordner fÃ¼r CSV sichergestellt/erstellt: {csv_output_directory}")
except Exception as e_mkdir:
    print(f"FEHLER beim Erstellen des CSV-Zielordners {csv_output_directory}: {e_mkdir}")
    raise
# --- Ende Ordnererstellung ---

# Sicherstellen, dass 'CellID' die erste Spalte ist, falls sie existiert und df nicht leer ist
if not df_features.empty and 'CellID' in df_features.columns:
    cols = ['CellID'] + [col for col in df_features.columns if col != 'CellID']
    df_features = df_features[cols]
elif df_features.empty:
    print("INFO: df_features ist leer. Es wird eine leere CSV-Datei gespeichert.")
else: # df_features nicht leer, aber 'CellID' fehlt
    print("WARNUNG: Spalte 'CellID' fehlt im finalen DataFrame, obwohl Daten vorhanden sind.")


print(f"Erste Zeilen der Feature-Tabelle ({df_features.shape}):\n{df_features.head()}")
df_features.to_csv(path_csv_output_for_quant, index=False)
print(f"Einzelzell-Feature-Tabelle gespeichert unter: {path_csv_output_for_quant}")
print("Zelle 2.5 (Manuelle Quantifizierung) erfolgreich ausgefÃ¼hrt.\n")

In [ ]:
# Zelle 2.75: PRE-FLIGHT CHECK - ÃœberprÃ¼fe alle Bedingungen vor CyLinter Start
print("=" * 80)
print("PRE-FLIGHT CHECK: ÃœberprÃ¼fe alle Voraussetzungen fÃ¼r CyLinter")
print("=" * 80)

import pandas as pd
import yaml
from pathlib import Path

# ZÃ¤hler fÃ¼r Erfolge und Fehler
checks_passed = 0
checks_failed = 0
warnings = 0

# --- CHECK 1: BenÃ¶tigte Variablen ---
print("\n[1] ÃœberprÃ¼fe benÃ¶tigte Variablen aus vorherigen Zellen...")
required_vars = ['BASE_DIR', 'CONFIG_PATH', 'MARKERS_CSV_PATH', 'all_channel_names_from_markers_csv', 'SAMPLE_NAME_FOR_FILES_QUANT']
for var_name in required_vars:
    if var_name in locals() or var_name in globals():
        print(f"  âœ“ {var_name} vorhanden")
        checks_passed += 1
    else:
        print(f"  âœ— FEHLER: {var_name} nicht gefunden! FÃ¼hren Sie vorherige Zellen aus.")
        checks_failed += 1

# --- CHECK 2: markers.csv Format ---
print("\n[2] ÃœberprÃ¼fe markers.csv Format...")
try:
    df_markers_check = pd.read_csv(MARKERS_CSV_PATH)
    print(f"  âœ“ markers.csv geladen: {len(df_markers_check)} EintrÃ¤ge")
    checks_passed += 1
    
    # WICHTIG: PrÃ¼fe dass markers.csv _cX Suffixe hat (fÃ¼r aggregateData)
    sample_markers = df_markers_check['marker_name'].head(5).tolist()
    has_cycle_suffix = any('_c' in str(m) for m in sample_markers)
    
    if not has_cycle_suffix:
        print(f"  âœ— FEHLER: Marker-Namen haben KEINE _cX Suffix: {sample_markers[:3]}")
        print(f"     â†’ markers.csv sollte _cX haben (fÃ¼r aggregateData CSV-Matching)")
        print(f"     â†’ Zelle 1 sollte dies automatisch korrigieren")
        checks_failed += 1
    else:
        print(f"  âœ“ Marker-Namen haben _cX (korrekt fÃ¼r aggregateData): {sample_markers[:3]}")
        checks_passed += 1
    
    # PrÃ¼fe auf Leerzeichen
    has_spaces = any(' ' in str(m) for m in df_markers_check['marker_name'])
    if has_spaces:
        print(f"  âš  WARNUNG: Leerzeichen in Marker-Namen gefunden")
        print(f"     â†’ Zelle 1 sollte diese entfernen")
        warnings += 1
    else:
        print(f"  âœ“ Keine Leerzeichen in Marker-Namen")
        checks_passed += 1
        
except Exception as e:
    print(f"  âœ— FEHLER beim Laden: {e}")
    checks_failed += 1

# --- CHECK 3: cylinter_config.yml ---
print("\n[3] ÃœberprÃ¼fe cylinter_config.yml...")
try:
    with open(CONFIG_PATH, 'r') as f:
        config = yaml.safe_load(f)
    print(f"  âœ“ Config geladen")
    checks_passed += 1
    
    # PrÃ¼fe sampleMetadata Format
    smd = config.get('sampleMetadata', {})
    if smd:
        sample_key = list(smd.keys())[0]
        sample_value = smd[sample_key]
        if isinstance(sample_value, list) and len(sample_value) == 5:
            print(f"  âœ“ sampleMetadata korrekt: {sample_key} -> 5-Element Liste")
            checks_passed += 1
        else:
            print(f"  âœ— FEHLER: sampleMetadata Format falsch: {sample_value}")
            print(f"     â†’ Muss 5-Element Liste sein: [name, condition, replicate, group, order]")
            checks_failed += 1
    else:
        print(f"  âœ— FEHLER: sampleMetadata fehlt")
        checks_failed += 1
    
    # PrÃ¼fe counterstainChannel
    counterstain = config.get('counterstainChannel')
    if counterstain:
        if counterstain in df_markers_check['marker_name'].values:
            print(f"  âœ“ counterstainChannel '{counterstain}' in markers.csv gefunden")
            checks_passed += 1
        else:
            print(f"  âœ— FEHLER: counterstainChannel '{counterstain}' NICHT in markers.csv!")
            print(f"     â†’ VerfÃ¼gbare DAPI Marker: {[m for m in df_markers_check['marker_name'] if 'DAPI' in str(m)][:3]}")
            checks_failed += 1
    else:
        print(f"  âœ— FEHLER: counterstainChannel fehlt in Config")
        checks_failed += 1
    
    # PrÃ¼fe samplesForROISelection
    roi_samples = config.get('samplesForROISelection', [])
    if roi_samples:
        roi_sample_name = roi_samples[0] if roi_samples else None
        if roi_sample_name == sample_key:
            print(f"  âœ“ samplesForROISelection stimmt mit sampleMetadata Ã¼berein: '{roi_sample_name}'")
            checks_passed += 1
        else:
            print(f"  âœ— FEHLER: samplesForROISelection '{roi_sample_name}' != sampleMetadata '{sample_key}'")
            checks_failed += 1
    else:
        print(f"  âš  WARNUNG: samplesForROISelection ist leer")
        warnings += 1
        
except Exception as e:
    print(f"  âœ— FEHLER beim Laden: {e}")
    checks_failed += 1

# --- CHECK 4: CSV Datei ---
print("\n[4] ÃœberprÃ¼fe erstellte CSV-Datei...")
try:
    csv_path = BASE_DIR / 'csv' / f"{SAMPLE_NAME_FOR_FILES_QUANT}.csv"
    if csv_path.exists():
        df_csv = pd.read_csv(csv_path)
        print(f"  âœ“ CSV gefunden: {csv_path.name}")
        print(f"    Zeilen: {len(df_csv)}, Spalten: {len(df_csv.columns)}")
        checks_passed += 1
        
        # PrÃ¼fe CellID
        if 'CellID' in df_csv.columns:
            print(f"  âœ“ CellID Spalte vorhanden")
            checks_passed += 1
        else:
            print(f"  âœ— FEHLER: CellID Spalte fehlt!")
            checks_failed += 1
        
        # KORRIGIERT: Vergleiche nur mit NICHT-EXCLUDIERTEN Markern!
        csv_marker_cols = [col for col in df_csv.columns if col not in ['CellID', 'Area', 'X_centroid', 'Y_centroid', 
                                                                          'MajorAxisLength', 'MinorAxisLength', 
                                                                          'Eccentricity', 'Solidity', 'Orientation']]
        
        # Hole excludierte Marker aus Config
        excluded_markers = config.get('markersToExclude', [])
        expected_markers = set(df_markers_check['marker_name'].values) - set(excluded_markers)
        csv_marker_set = set(csv_marker_cols)
        
        if csv_marker_set == expected_markers:
            print(f"  âœ“ CSV Spalten stimmen mit erwarteten Markern Ã¼berein ({len(csv_marker_set)} Marker)")
            print(f"    ({len(excluded_markers)} Marker ausgeschlossen wie konfiguriert)")
            checks_passed += 1
        else:
            missing_in_csv = expected_markers - csv_marker_set
            extra_in_csv = csv_marker_set - expected_markers
            if missing_in_csv:
                print(f"  âœ— FEHLER: {len(missing_in_csv)} erwartete Marker fehlen in CSV")
                print(f"     Erste 3: {list(missing_in_csv)[:3]}")
                checks_failed += 1
            if extra_in_csv:
                print(f"  âš  WARNUNG: {len(extra_in_csv)} unerwartete Marker in CSV")
                print(f"     Erste 3: {list(extra_in_csv)[:3]}")
                warnings += 1
            if not missing_in_csv and not extra_in_csv:
                checks_passed += 1
                
    else:
        print(f"  âœ— FEHLER: CSV nicht gefunden: {csv_path}")
        print(f"     â†’ FÃ¼hren Sie Zelle 2.5 aus")
        checks_failed += 1
        
except Exception as e:
    print(f"  âœ— FEHLER: {e}")
    checks_failed += 1

# --- CHECK 5: DateizÃ¤hlung (KORRIGIERT: ignoriere .DS_Store) ---
print("\n[5] ÃœberprÃ¼fe DateizÃ¤hlung in Ordnern...")
folders_to_check = ['csv', 'tif', 'seg', 'mask']
file_counts = {}
for folder in folders_to_check:
    folder_path = BASE_DIR / folder
    if folder_path.exists():
        # Ignoriere .DS_Store und andere versteckte Dateien
        files = [f for f in folder_path.glob('*') if f.is_file() and not f.name.startswith('.')]
        file_counts[folder] = len(files)
    else:
        file_counts[folder] = 0

all_counts_equal = len(set(file_counts.values())) == 1
if all_counts_equal and file_counts['csv'] > 0:
    print(f"  âœ“ Alle Ordner haben gleiche Anzahl Dateien: {file_counts}")
    checks_passed += 1
else:
    print(f"  âœ— FEHLER: Unterschiedliche Dateianzahl: {file_counts}")
    print(f"     â†’ Alle Ordner mÃ¼ssen GENAU 1 Datei haben mit gleichem Namen")
    checks_failed += 1

# --- ZUSAMMENFASSUNG ---
print("\n" + "=" * 80)
print("ZUSAMMENFASSUNG PRE-FLIGHT CHECK")
print("=" * 80)
print(f"âœ“ Erfolgreich: {checks_passed}")
print(f"âœ— Fehler:      {checks_failed}")
print(f"âš  Warnungen:   {warnings}")
print()

if checks_failed == 0:
    print("ðŸŽ‰ ALLE CHECKS BESTANDEN! CyLinter kann gestartet werden (Zelle 8).")
    print()
    print("NÃ„CHSTE SCHRITTE:")
    print("  1. FÃ¼hren Sie Zelle 8 aus (Konfigurations-GUI)")
    print("  2. FÃ¼hren Sie Zelle 9 aus (Pipeline-Start)")
else:
    print("âŒ FEHLER GEFUNDEN! Bitte beheben Sie die Probleme vor CyLinter Start.")
    print()

print("=" * 80)
print("Pre-Flight Check abgeschlossen.\n")

---

## ?? Zelle 8: Konfigurations-GUI (Tag 1)

Marker-Subset ausw?hlen (Schnellauswahl oder Custom). Speichert nur `markersToExclude` in `cylinter_config.yml`; Pipeline startet in Zelle 9.


In [ ]:
# Zelle 8: Konfigurations-GUI (TAG 1 - Erste Analyse)
import ipywidgets as widgets
from IPython.display import display
import yaml
from pathlib import Path
import pandas as pd

print("="*80)
print("KONFIGURATIONS-GUI (TAG 1 - ERSTE ANALYSE)")
print("="*80)

# ÃœberprÃ¼fe benÃ¶tigte Variablen
if 'BASE_DIR' not in locals():
    BASE_DIR = Path.cwd()
if 'CONFIG_PATH' not in locals():
    CONFIG_PATH = BASE_DIR / "cylinter_config.yml"
if 'MARKERS_CSV_PATH' not in locals():
    MARKERS_CSV_PATH = BASE_DIR / "markers.csv"

# Lade markers.csv
df_markers = pd.read_csv(MARKERS_CSV_PATH)

# Kategorisiere Marker
dapi_markers = df_markers[df_markers['marker_name'].str.contains('DAPI', case=False, na=False)]['marker_name'].tolist()
af_markers = df_markers[df_markers['marker_name'].str.contains('AF[12]_', case=True, na=False, regex=True)]['marker_name'].tolist()
bio_markers = df_markers[
    ~df_markers['marker_name'].str.contains('DAPI', case=False, na=False) &
    ~df_markers['marker_name'].str.contains('AF[12]_', case=True, na=False, regex=True)
]['marker_name'].tolist()

print(f"Marker geladen: {len(dapi_markers)} DAPI, {len(af_markers)} AF, {len(bio_markers)} Bio")

# Quick Options
quick_label = widgets.HTML("<h4>Schnellauswahl:</h4>")
quick_option = widgets.RadioButtons(
    options=[
        ('Alle 96 Marker verwenden', 'all'),
        ('Alle ausser AF (58 Marker: DAPI + Bio)', 'no_af'),
        ('Nur Bio-Marker (ohne DAPI/AF)', 'bio_only'),
        ('Custom (unten auswaehlen)', 'custom')
    ],
    value='no_af',
    layout=widgets.Layout(width='400px')
)

quick_info = widgets.HTML("""
<div style="background-color: #e3f2fd; padding: 10px; border-left: 4px solid #2196F3; margin: 10px 0;">
<b>Empfehlung:</b> "Alle ausser AF" (58 Marker)<br>
Autofluoreszenz-Marker werden automatisch ausgeschlossen.
</div>
""")

# Bio-Marker Checkboxen
bio_checkboxes = {}
bio_checkbox_widgets = []

for marker in sorted(bio_markers):
    cb = widgets.Checkbox(
        value=True,
        description=marker,
        indent=False,
        layout=widgets.Layout(width='220px'),
        style={'description_width': 'initial'}
    )
    bio_checkboxes[marker] = cb
    bio_checkbox_widgets.append(cb)

bio_grid = widgets.GridBox(bio_checkbox_widgets, layout=widgets.Layout(
    grid_template_columns='repeat(3, 220px)',
    grid_gap='5px',
    margin='10px 0',
    max_height='400px',
    overflow_y='auto'
))

# Bio-Marker Controls
bio_select_all = widgets.Button(description='Alle', layout=widgets.Layout(width='120px'))
bio_deselect_all = widgets.Button(description='Keine', layout=widgets.Layout(width='120px'))

def bio_select_all_click(_):
    for cb in bio_checkboxes.values():
        cb.value = True

def bio_deselect_all_click(_):
    for cb in bio_checkboxes.values():
        cb.value = False

bio_select_all.on_click(bio_select_all_click)
bio_deselect_all.on_click(bio_deselect_all_click)

bio_controls = widgets.HBox([bio_select_all, bio_deselect_all])
detail_label = widgets.HTML("<h4>Custom: Bio-Marker einzeln auswaehlen</h4>")

# Status Output
status_output = widgets.Output(layout=widgets.Layout(
    border='1px solid #ccc', padding='10px', margin='10px 0',
    max_height='300px', overflow_y='auto'
))

# Save Function
def save_and_ready(b):
    """Speichert nur Marker-Auswahl - Pipeline wird in Zelle 9 gestartet"""
    with status_output:
        status_output.clear_output()
        
        print("\n" + "="*70)
        print("SPEICHERE MARKER-KONFIGURATION")
        print("="*70 + "\n")
        
        # Marker-Auswahl
        option = quick_option.value
        markers_to_exclude = []
        
        if option == 'all':
            markers_to_exclude = []
            marker_desc = "Alle 96 Marker"
        elif option == 'no_af':
            markers_to_exclude = af_markers
            marker_desc = f"58 Marker (DAPI + Bio, ohne {len(af_markers)} AF)"
        elif option == 'bio_only':
            markers_to_exclude = dapi_markers + af_markers
            marker_desc = f"{len(bio_markers)} Bio-Marker (ohne DAPI/AF)"
        elif option == 'custom':
            excluded_bio = [m for m, cb in bio_checkboxes.items() if not cb.value]
            markers_to_exclude = af_markers + excluded_bio
            marker_desc = f"{len(dapi_markers) + len(bio_markers) - len(excluded_bio)} Marker (Custom)"
        
        print(f"Marker-Auswahl: {marker_desc}")
        print(f"   Ausgeschlossen: {len(markers_to_exclude)} Marker")
        print(f"   Werden verwendet: {96 - len(markers_to_exclude)} Marker")
        
        # Schreibe Config
        try:
            # Backup
            backup_path = CONFIG_PATH.with_suffix('.yml.backup')
            import shutil
            shutil.copy2(CONFIG_PATH, backup_path)
            print(f"\nBackup erstellt: {backup_path.name}")
            
            # Lade Config
            with open(CONFIG_PATH, 'r') as f:
                config = yaml.safe_load(f)
            
            # Update markersToExclude
            config['markersToExclude'] = markers_to_exclude
            
            # Schreibe Config
            temp_path = CONFIG_PATH.with_suffix('.yml.tmp')
            with open(temp_path, 'w') as f:
                yaml.dump(config, f, default_flow_style=False, sort_keys=False)
            temp_path.replace(CONFIG_PATH)
            
            print(f"cylinter_config.yml aktualisiert")
            
            print("\n" + "="*70)
            print("KONFIGURATION GESPEICHERT!")
            print("="*70 + "\n")
            
            print("NAECHSTER SCHRITT:")
            print("   Fuehren Sie ZELLE 9 aus, um die Pipeline zu starten!")
            print("   Die Pipeline laeuft von aggregateData bis zum Ende durch.")
            print("   Module mit existierenden Checkpoints werden uebersprungen.\n")
            
        except Exception as e:
            print(f"\nFEHLER: {e}")
            import traceback
            traceback.print_exc()

save_btn = widgets.Button(
    description='Speichern & Bereit',
    button_style='success',
    layout=widgets.Layout(width='220px', height='45px'),
    icon='check',
    tooltip='Speichert Marker-Konfiguration (startet KEINE Pipeline!)'
)
save_btn.on_click(save_and_ready)

# UI
info_box = widgets.HTML("""
<div style="background-color: #fff3cd; padding: 15px; border-left: 4px solid #ffc107; margin-bottom: 15px;">
<b>Funktionsweise:</b><br>
Diese GUI konfiguriert nur die <b>Marker-Auswahl</b><br>
Die Pipeline startet in <b>Zelle 9</b><br>
CyLinter laeuft automatisch durch alle Module von <b>aggregateData bis zum Ende</b><br>
Module mit existierenden Checkpoints werden <b>automatisch uebersprungen</b><br><br>
<b>Fuer TAG 1 (erste Analyse):</b> Waehlen Sie Marker, Speichern, Zelle 9 ausfuehren<br>
<b>Fuer TAG 2+ (neue Marker):</b> Verwenden Sie Zelle 13 (Erweiterte Steuerung)
</div>
""")

ui = widgets.VBox([
    widgets.HTML("<h3>Marker-Konfiguration</h3>"),
    info_box,
    quick_label,
    quick_option,
    quick_info,
    detail_label,
    bio_controls,
    bio_grid,
    widgets.HTML("<hr>"),
    save_btn,
    status_output
])

display(ui)

print("\nKonfigurations-GUI bereit!")
print("   Waehlen Sie Marker-Strategie und klicken Sie 'Speichern & Bereit'")
print("   Pipeline wird dann in Zelle 9 gestartet.\n")

In [ ]:
# Zelle 9: Pipeline-Start (TAG 1) - MIT INTELLIGENTER CHECKPOINT-ERKENNUNG

import ipywidgets as widgets
from IPython.display import display
import subprocess
import os
from pathlib import Path
import yaml
import time

print("="*80)
print("CYLINTER PIPELINE-START (TAG 1)")
print("="*80)
print("Intelligente Checkpoint-Erkennung: Startet bei erstem fehlenden Modul!")
print("="*80)

# Ueberpruefe benoetigte Variablen
if 'BASE_DIR' not in locals():
    BASE_DIR = Path.cwd()
if 'CONFIG_PATH' not in locals():
    CONFIG_PATH = BASE_DIR / "cylinter_config.yml"
if 'CYLINTER_ENV_PATH' not in locals():
    CYLINTER_ENV_PATH = Path("/opt/anaconda3/envs/cylinter_ben")
if 'CHECKPOINT_DIR' not in locals():
    CHECKPOINT_DIR = BASE_DIR / "cylinter_output_prune_test" / "checkpoints"

# CyLinter Executables
cylinter_executable = CYLINTER_ENV_PATH / "bin" / "cylinter"
python_executable = CYLINTER_ENV_PATH / "bin" / "python"

# Alle Module in Pipeline-Reihenfolge
ALL_MODULES = [
    "aggregateData", "selectROIs", "intensityFilter", "areaFilter",
    "cycleCorrelation", "logTransform", "pruneOutliers", "metaQC", "PCA",
    "setContrast", "gating", "clustering", "clustermap", "frequencyStats", "curateThumbnails"
]

# Output Widget
output_widget = widgets.Output(layout=widgets.Layout(
    border='1px solid #ccc', padding='10px', margin='10px 0',
    max_height='600px', overflow_y='auto'
))

# Pruefe ob Config existiert
if not CONFIG_PATH.exists():
    error_box = widgets.HTML("""
    <div style="background-color: #ffebee; padding: 20px; border-left: 4px solid #f44336;">
    <h3>FEHLER: Konfiguration fehlt!</h3>
    <p>Die Datei <code>cylinter_config.yml</code> wurde nicht gefunden.</p>
    <p><b>Bitte fuehren Sie zuerst ZELLE 8 aus!</b></p>
    </div>
    """)
    ui = widgets.VBox([error_box])
    display(ui)
    
else:
    # Config vorhanden - analysiere Checkpoints
    with open(CONFIG_PATH, 'r') as f:
        config = yaml.safe_load(f)
    
    markers_excluded = len(config.get('markersToExclude', []))
    
    # INTELLIGENTE CHECKPOINT-ERKENNUNG
    CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)
    existing_checkpoints = []
    missing_checkpoints = []
    
    for module in ALL_MODULES:
        checkpoint_file = CHECKPOINT_DIR / f"{module}.parquet"
        if checkpoint_file.exists():
            existing_checkpoints.append(module)
        else:
            missing_checkpoints.append(module)
    
    # Bestimme Startmodul
    if len(missing_checkpoints) == 0:
        suggested_start = None
        status_message = "ALLE CHECKPOINTS VORHANDEN! Pipeline komplett."
        status_color = "#4caf50"
    else:
        suggested_start = missing_checkpoints[0]
        status_message = f"Erstes fehlendes Checkpoint: {suggested_start}"
        status_color = "#ff9800"
    
    # Zeige Status
    print("\n" + "="*70)
    print("CHECKPOINT-ANALYSE:")
    print("="*70 + "\n")
    print(f"Existierende Checkpoints: {len(existing_checkpoints)}/15")
    for cp in existing_checkpoints:
        print(f"   [OK] {cp}")
    print(f"\nFehlende Checkpoints: {len(missing_checkpoints)}/15")
    for cp in missing_checkpoints:
        print(f"   [--] {cp}")
    print()
    
    # Pipeline-Start-Funktion
    def start_pipeline(b):
        """Startet die Pipeline beim ersten fehlenden Modul"""
        with output_widget:
            output_widget.clear_output()
            
            # Re-check Checkpoints (falls sich was geaendert hat)
            missing = []
            for module in ALL_MODULES:
                if not (CHECKPOINT_DIR / f"{module}.parquet").exists():
                    missing.append(module)
            
            if len(missing) == 0:
                print("\n" + "="*70)
                print("ALLE CHECKPOINTS VORHANDEN!")
                print("="*70 + "\n")
                print("Die Pipeline ist komplett durchgelaufen.")
                print("Falls Sie Module neu berechnen wollen, nutzen Sie Zelle 12.")
                return
            
            start_module = missing[0]
            
            print("\n" + "="*70)
            print(f"STARTE PIPELINE BEI: {start_module}")
            print("="*70 + "\n")
            print("Pipeline-Reihenfolge:")
            for i, mod in enumerate(ALL_MODULES, 1):
                status = "[SKIP]" if mod in existing_checkpoints else "[RUN!]"
                marker = ">>>" if mod == start_module else "   "
                print(f"{marker} {i:2}. {status} {mod}")
            print()
            
            # WICHTIG: Wenn start_module NICHT aggregateData ist, aber auch nicht
            # das vorherige Modul hat, muessen wir bei aggregateData starten!
            if start_module != "aggregateData":
                module_index = ALL_MODULES.index(start_module)
                previous_module = ALL_MODULES[module_index - 1]
                previous_checkpoint = CHECKPOINT_DIR / f"{previous_module}.parquet"
                
                if not previous_checkpoint.exists():
                    print(f"[!] WARNUNG: Vorheriges Checkpoint '{previous_module}.parquet' fehlt!")
                    print(f"[!] Muss bei 'aggregateData' starten statt '{start_module}'!\n")
                    start_module = "aggregateData"
            
            if cylinter_executable.exists():
                cmd = [str(cylinter_executable), "--module", start_module, str(CONFIG_PATH)]
                print(f"[*] Command: cylinter --module {start_module} config.yml")
            else:
                cmd = [str(python_executable), "-m", "cylinter.cylinter", "--module", start_module, str(CONFIG_PATH)]
                print(f"[*] Command: python -m cylinter.cylinter --module {start_module} config.yml")
            
            print(f"\n[!] Pipeline laeuft von '{start_module}' bis zum Ende!")
            print("[!] Interaktive Module (selectROIs, setContrast, gating) oeffnen GUIs\n")
            
            overall_start = time.time()
            
            try:
                env = os.environ.copy()
                process = subprocess.Popen(
                    cmd, cwd=str(BASE_DIR), env=env,
                    stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                    universal_newlines=True, bufsize=1
                )
                
                # Stream Output
                for line in process.stdout:
                    print(line, end='')
                
                return_code = process.wait()
                total_duration = time.time() - overall_start
                
                # Zusammenfassung
                print("\n" + "="*70)
                if return_code == 0:
                    print("PIPELINE ERFOLGREICH!")
                    print(f"   Laufzeit: {total_duration:.1f}s ({total_duration/60:.1f} Min)")
                else:
                    print(f"PIPELINE FEHLER (Exit Code: {return_code})")
                    print(f"   Laufzeit: {total_duration:.1f}s ({total_duration/60:.1f} Min)")
                print("="*70 + "\n")
                
                # Zeige welche Checkpoints jetzt existieren
                print("Checkpoint-Status nach Run:")
                for module in ALL_MODULES:
                    cp_file = CHECKPOINT_DIR / f"{module}.parquet"
                    status = "[OK]" if cp_file.exists() else "[--]"
                    print(f"   {status} {module}")
                    
            except Exception as e:
                print(f"\n[!] FEHLER beim Pipeline-Start: {e}")
                import traceback
                traceback.print_exc()
    
    # Start Button
    start_btn = widgets.Button(
        description='Pipeline starten',
        button_style='success' if suggested_start else 'primary',
        layout=widgets.Layout(width='200px', height='50px'),
        icon='play',
        tooltip=f'Startet bei: {suggested_start}' if suggested_start else 'Pipeline komplett'
    )
    start_btn.on_click(start_pipeline)
    
    # Info Box
    info_html = f"""
    <div style="background-color: {status_color}22; padding: 15px; border-left: 4px solid {status_color};">
    <b>Status:</b> {status_message}<br><br>
    <b>Konfiguration:</b><br>
    {96 - markers_excluded} Marker werden verwendet<br><br>
    <b>Existierende Checkpoints:</b> {len(existing_checkpoints)}/15<br>
    <b>Fehlende Checkpoints:</b> {len(missing_checkpoints)}/15<br><br>
    """
    
    if suggested_start:
        info_html += f"""
        <b>Naechster Schritt:</b><br>
        Pipeline wird bei <b>{suggested_start}</b> starten<br>
        und bis zum Ende durchlaufen.<br><br>
        """
    else:
        info_html += """
        <b>Pipeline komplett!</b><br>
        Alle Module wurden bereits ausgefuehrt.<br><br>
        """
    
    info_html += """
    <b>WICHTIG:</b><br>
    CyLinter ueberschreibt alle Checkpoints ab dem Startmodul!<br>
    Wenn Sie einzelne Module neu berechnen wollen, nutzen Sie Zelle 12.
    </div>
    """
    
    info_box = widgets.HTML(info_html)
    
    # UI
    ui = widgets.VBox([
        widgets.HTML("<h3>Pipeline-Start (Intelligente Checkpoint-Erkennung)</h3>"),
        info_box,
        start_btn,
        widgets.HTML("<hr><h4>Pipeline-Output:</h4>"),
        output_widget
    ])
    
    display(ui)
    
    print("\nPipeline-Start bereit!")
    if suggested_start:
        print(f"   Klicken Sie 'Pipeline starten' um bei '{suggested_start}' zu beginnen.\n")
    else:
        print("   Pipeline ist bereits komplett!\n")

In [ ]:
# STOP-Zelle - TAG 1 ENDET HIER!

import sys
from IPython.display import display, HTML

# Zeige STOP-Warnung
stop_message = HTML("""
<div style="background-color: #fff3cd; padding: 30px; border: 4px solid #ff9800; text-align: center;">
<h1 style="color: #f57c00; margin: 0;">STOP - TAG 1 ENDET HIER!</h1>
<hr>
<h3>Sie haben die GUIs fuer TAG 1 geladen:</h3>
<p style="font-size: 18px;">
[+] Zelle 8: Marker-Konfiguration<br>
[+] Zelle 9: Pipeline-Start
</p>
<hr>
<h3>NAECHSTE SCHRITTE:</h3>
<ol style="font-size: 16px; text-align: left; max-width: 600px; margin: auto;">
<li><b>Zelle 8:</b> Marker auswaehlen â†’ "Speichern & Bereit"</li>
<li><b>Zelle 9:</b> "Pipeline starten" klicken</li>
<li>Warten Sie bis Pipeline komplett durchgelaufen ist</li>
</ol>
<hr>
<h3 style="color: #d32f2f;">WARNUNG:</h3>
<p style="font-size: 16px;">
Die Zellen UNTERHALB sind fuer TAG 2+ (Folge-Sessions)!<br>
<b>Fuehren Sie diese NICHT aus waehrend TAG 1 laeuft!</b>
</p>
<hr>
<p style="font-size: 14px; color: #666;">
Falls Sie "Run All" benutzt haben: Das ist okay!<br>
Diese Zelle stoppt die automatische Ausfuehrung hier.
</p>
</div>
""")

display(stop_message)

print("\n" + "="*80)
print("TAG 1 WORKFLOW BEREIT!")
print("="*80)
print("\nGehen Sie zu:")
print("  1. Zelle 8: Marker konfigurieren")
print("  2. Zelle 9: Pipeline starten")
print("\nDie Zellen unterhalb (12-13) sind fuer TAG 2+ (spaetere Sessions).")
print("="*80 + "\n")

# WICHTIG: Verhindere weiteres "Run All"
# Kommentieren Sie diese Zeile aus, wenn Sie Zelle 12 ausfuehren wollen:
raise SystemExit("STOP: TAG 1 Setup komplett. Verwenden Sie Zelle 8 und 9.")

---

## ?? Zelle 12: Erweiterte Pipeline-Steuerung (TAG 2+ oder Fehlerkorrektur)

**?? NUR VERWENDEN WENN:**

**Szenario A - Neue Marker hinzuf?gen (TAG 2+):**
- Sie haben TAG 1 komplett abgeschlossen
- Sie wollen zus?tzliche Marker hinzuf?gen
- Alte Gating-Ergebnisse sollen erhalten bleiben

**Szenario B - Modul neu starten (Fehlerkorrektur):**
- Ein Modul ist fehlgeschlagen oder Ergebnis ist nicht korrekt
- Sie wollen nur dieses Modul (und nachfolgende) neu laufen lassen
- Vorherige Module sollen NICHT neu berechnet werden

---

**Diese GUI bietet:**
- ? **Neue Marker extrahieren & mergen** (wie Zelle 12 vorher)
- ? **Vollst?ndige Pipeline-Modul-Kontrolle** (wie Zelle 8)
- ? **Checkpoint-Reset f?r einzelne Module** (z.B. nur `gating` neu starten)

---

**Workflow TAG 2+ (Neue Marker):**
```
1. Kernel ? Restart
2. Nur Zellen 2-3 ausf?hren
3. Diese Zelle ausf?hren ? GUI ?ffnet sich
4. Tab 1: Neue Marker ausw?hlen ? "Extract & Merge"
5. Tab 2: Nur "aggregateData" + "gating" ausw?hlen ? "Run Selected"
```

**Workflow Fehlerkorrektur (Modul neu starten):**
```
1. Kernel ? Restart
2. Nur Zellen 2-3 ausf?hren
3. Diese Zelle ausf?hren ? GUI ?ffnet sich
4. Tab 2: "Reset Checkpoint" f?r fehlerhaftes Modul klicken
5. Tab 2: Fehlerhaftes Modul + nachfolgende ausw?hlen ? "Run Selected"
```

**?? Bei ERSTER SESSION (TAG 1): Verwenden Sie Zelle 8 + Zelle 10!**


In [ ]:
# Zelle 12: Erweiterte Pipeline-Steuerung (TAG 2+ / Fehlerkorrektur) - MIT INTELLIGENTER CHECKPOINT-ERKENNUNG

# IMPORTS FIRST!
import ipywidgets as widgets
from IPython.display import display
import subprocess
import os
from pathlib import Path
import yaml
import time
import pandas as pd
import numpy as np
import tifffile
from skimage.measure import regionprops_table

print("="*80)
print("ERWEITERTE PIPELINE-STEUERUNG (TAG 2+ / FEHLERKORREKTUR)")
print("="*80)
print("Mit intelligenter Checkpoint-Erkennung!")
print("="*80)

# Ueberpruefe benoetigte Variablen
if 'BASE_DIR' not in locals():
    BASE_DIR = Path.cwd()
if 'CONFIG_PATH' not in locals():
    CONFIG_PATH = BASE_DIR / "cylinter_config.yml"
if 'CYLINTER_ENV_PATH' not in locals():
    CYLINTER_ENV_PATH = Path("/opt/anaconda3/envs/cylinter_ben")
if 'CHECKPOINT_DIR' not in locals():
    CHECKPOINT_DIR = BASE_DIR / "cylinter_output_prune_test" / "checkpoints"
if 'MARKERS_CSV_PATH' not in locals():
    MARKERS_CSV_PATH = BASE_DIR / "markers.csv"

# CyLinter Executables
cylinter_executable = CYLINTER_ENV_PATH / "bin" / "cylinter"
python_executable = CYLINTER_ENV_PATH / "bin" / "python"

# Alle Module (Pipeline-Reihenfolge)
ALL_MODULES = [
    "aggregateData", "selectROIs", "intensityFilter", "areaFilter",
    "cycleCorrelation", "logTransform", "pruneOutliers", "metaQC", "PCA",
    "setContrast", "gating", "clustering", "clustermap", "frequencyStats", "curateThumbnails"
]

# ============================================================================
# TAB 1: NEUE MARKER EXTRAHIEREN & MERGEN (TAG 2)
# ============================================================================

# Lade markers.csv
df_markers = pd.read_csv(MARKERS_CSV_PATH)
bio_markers = df_markers[
    ~df_markers['marker_name'].str.contains('DAPI', case=False, na=False) &
    ~df_markers['marker_name'].str.contains('AF[12]_', case=True, na=False, regex=True)
]['marker_name'].tolist()

# Output fuer Marker-Extraktion (KEINE Hoehenbegrenzung!)
marker_output = widgets.Output(layout=widgets.Layout(
    border='1px solid #ddd', padding='10px', margin='10px 0'
))

# Checkboxen fuer Marker
bio_checkboxes = {}
bio_checkbox_widgets = []
for marker in sorted(bio_markers):
    cb = widgets.Checkbox(value=False, description=marker, indent=False,
                          layout=widgets.Layout(width='220px'))
    bio_checkboxes[marker] = cb
    bio_checkbox_widgets.append(cb)

bio_grid = widgets.GridBox(bio_checkbox_widgets, layout=widgets.Layout(
    grid_template_columns='repeat(3, 220px)', grid_gap='5px',
    margin='10px 0'
))

# Buttons fuer Marker-Tab
marker_select_all = widgets.Button(description='[*] Select All', layout=widgets.Layout(width='150px'))
marker_deselect_all = widgets.Button(description='[ ] Deselect All', layout=widgets.Layout(width='150px'))

def marker_select_all_click(_):
    for cb in bio_checkboxes.values():
        cb.value = True

def marker_deselect_all_click(_):
    for cb in bio_checkboxes.values():
        cb.value = False

marker_select_all.on_click(marker_select_all_click)
marker_deselect_all.on_click(marker_deselect_all_click)

marker_controls = widgets.HBox([marker_select_all, marker_deselect_all])

# Extract & Merge Button
extract_btn = widgets.Button(
    description='[>] Extract & Merge New Markers',
    button_style='success',
    layout=widgets.Layout(width='280px', height='45px'),
    icon='flask'
)

def extract_and_merge_markers(b):
    """Extrahiert neue Marker und merged sie mit bestehender CSV"""
    with marker_output:
        marker_output.clear_output()
        
        selected_markers = [marker for marker, cb in bio_checkboxes.items() if cb.value]
        
        if not selected_markers:
            print("WARNUNG: KEINE MARKER AUSGEWAEHLT!")
            return
        
        print(f"\n{'='*70}")
        print(f"MARKER-EXTRAKTION & MERGE")
        print(f"{'='*70}\n")
        print(f"Ausgewaehlte Marker: {len(selected_markers)}")
        
        try:
            # Lade Config
            with open(CONFIG_PATH, 'r') as f:
                config = yaml.safe_load(f)
            
            inDir = BASE_DIR / config.get('inDir', '.')
            csv_dir = config.get('csv_dir', 'csv')
            smd = config['sampleMetadata']
            sample_name = list(smd.values())[0][0]
            
            csv_path = inDir / csv_dir / f"{sample_name}.csv"
            tif_path = inDir / 'tif' / f"{sample_name}.ome.tif"
            mask_path = inDir / 'mask' / f"{sample_name}.tif"
            
            print(f"[+] Sample: {sample_name}")
            
            # Lade alte CSV
            if not csv_path.exists():
                print(f"[!] CSV nicht gefunden: {csv_path}")
                return
            
            old_csv = pd.read_csv(csv_path)
            print(f"[+] Alte CSV geladen: {len(old_csv.columns)} Spalten")
            
            # Extrahiere neue Marker aus TIF
            print(f"\n[*] Extrahiere {len(selected_markers)} neue Marker...")
            
            tif_data = tifffile.imread(tif_path)
            mask_data = tifffile.imread(mask_path)
            
            marker_indices = [i for i, m in enumerate(df_markers['marker_name']) if m in selected_markers]
            
            print(f"[+] TIF Shape: {tif_data.shape}")
            print(f"[+] Mask Shape: {mask_data.shape}")
            
            new_data = {}
            for idx, marker in zip(marker_indices, selected_markers):
                channel_data = tif_data[idx]
                props = regionprops_table(mask_data, intensity_image=channel_data,
                                         properties=['label', 'mean_intensity'])
                new_data[marker] = dict(zip(props['label'], props['mean_intensity']))
                print(f"  [+] {marker}: {len(props['label'])} Zellen")
            
            # Merge mit alter CSV
            print(f"\n[*] Merge mit bestehender CSV...")
            
            new_columns_df = pd.DataFrame(new_data)
            new_columns_df.index = old_csv.index
            
            merged_csv = pd.concat([old_csv, new_columns_df], axis=1)
            
            print(f"[+] Alte CSV: {len(old_csv.columns)} Spalten")
            print(f"[+] Neue Spalten: {len(selected_markers)}")
            print(f"[+] Merged CSV: {len(merged_csv.columns)} Spalten")
            
            # Backup & Save
            from datetime import datetime
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            backup_path = csv_path.with_suffix(f'.csv.backup_{timestamp}')
            
            import shutil
            shutil.copy2(csv_path, backup_path)
            print(f"[+] Backup: {backup_path.name}")
            
            merged_csv.to_csv(csv_path, index=False)
            print(f"[+] CSV gespeichert: {csv_path.name}")
            
            print(f"\n{'='*70}")
            print("ERFOLGREICH!")
            print(f"{'='*70}\n")
            print(f"[>] NAECHSTER SCHRITT:")
            print(f"   1. Tab 2: Loesche aggregateData.parquet")
            print(f"   2. Tab 2: Starte Pipeline bei aggregateData\n")
            
        except Exception as e:
            print(f"\n[!] FEHLER: {e}")
            import traceback
            traceback.print_exc()

extract_btn.on_click(extract_and_merge_markers)

# Tab 1 Layout mit Scrollbar
tab1_title = widgets.HTML("<h3>Neue Marker hinzufuegen (TAG 2+)</h3>")
tab1_info = widgets.HTML("""
<div style="background-color: #fff3cd; padding: 10px; border-left: 4px solid #ffc107;">
<b>Zweck:</b> Zusaetzliche Marker zur bestehenden CSV hinzufuegen<br><br>
<b>Workflow:</b><br>
1. Waehlen Sie neue Marker aus<br>
2. Klicken Sie "Extract & Merge"<br>
3. Gehen Sie zu Tab 2 â†’ Loeschen Sie aggregateData Checkpoint<br>
4. Starten Sie Pipeline bei aggregateData
</div>
""")

tab1_content = widgets.VBox([
    tab1_title,
    tab1_info,
    marker_controls,
    bio_grid,
    extract_btn,
    marker_output
])  # KEINE Hoehenbegrenzung mehr!

# ============================================================================
# TAB 2: CHECKPOINT-RESET & INTELLIGENTE PIPELINE-START
# ============================================================================

# Output fuer Pipeline (KEINE Hoehenbegrenzung!)
pipeline_output = widgets.Output(layout=widgets.Layout(
    border='1px solid #ccc', padding='10px', margin='10px 0'
))

# Checkpoint Reset Buttons (fuer jedes Modul)
reset_buttons = {}
reset_button_widgets = []

CHECKPOINT_DIR.mkdir(parents=True, exist_ok=True)

# Report Path fuer Gating-Reset
REPORT_PATH = BASE_DIR / "cylinter_output_prune_test" / "cylinter_report.yml"

for module in ALL_MODULES:
    btn = widgets.Button(
        description=f'[X] {module}',
        button_style='danger',
        layout=widgets.Layout(width='220px'),
        tooltip=f'Loescht Checkpoint fuer {module}' if module != 'gating' else 'Loescht Checkpoint UND Report-Daten fuer Gating'
    )
    
    def make_reset_handler(mod):
        def handler(_):
            with pipeline_output:
                checkpoint_file = CHECKPOINT_DIR / f"{mod}.parquet"
                
                # Loesche Checkpoint
                if checkpoint_file.exists():
                    checkpoint_file.unlink()
                    print(f"[+] Checkpoint geloescht: {mod}.parquet")
                    print(f"  -> Modul '{mod}' wird beim naechsten Run neu berechnet")
                else:
                    print(f"[i] Kein Checkpoint vorhanden: {mod}.parquet")
                
                # SPEZIAL: Fuer Gating auch Report-Daten loeschen!
                if mod == 'gating' and REPORT_PATH.exists():
                    try:
                        import yaml
                        with open(REPORT_PATH, 'r') as f:
                            report = yaml.safe_load(f)
                        
                        if 'gating' in report and report['gating']:
                            num_gates = len(report['gating'])
                            report['gating'] = {}
                            
                            with open(REPORT_PATH, 'w') as f:
                                yaml.dump(report, f, default_flow_style=False)
                            
                            print(f"[+] {num_gates} Gating-Eintraege aus Report geloescht")
                            print(f"  -> Gating-GUI wird sich beim naechsten Run oeffnen!")
                        else:
                            print(f"[i] Keine Gating-Daten im Report vorhanden")
                    except Exception as e:
                        print(f"[!] Fehler beim Loeschen der Report-Daten: {e}")
                
                print()
        return handler
    
    btn.on_click(make_reset_handler(module))
    reset_buttons[module] = btn
    reset_button_widgets.append(btn)

reset_grid = widgets.GridBox(reset_button_widgets, layout=widgets.Layout(
    grid_template_columns='repeat(3, 220px)',
    grid_gap='10px',
    margin='10px 0'
))

# INTELLIGENTE CHECKPOINT-ERKENNUNG
existing_checkpoints = []
missing_checkpoints = []

for module in ALL_MODULES:
    checkpoint_file = CHECKPOINT_DIR / f"{module}.parquet"
    if checkpoint_file.exists():
        existing_checkpoints.append(module)
    else:
        missing_checkpoints.append(module)

# Bestimme Vorschlag
if len(missing_checkpoints) == 0:
    suggested_start = ALL_MODULES[0]  # aggregateData als Default
else:
    suggested_start = missing_checkpoints[0]

# Dropdown fuer Modul-Start (mit Vorschlag)
module_dropdown = widgets.Dropdown(
    options=ALL_MODULES,
    value=suggested_start,
    description='Start bei:',
    layout=widgets.Layout(width='300px'),
    style={'description_width': '80px'}
)

# Pipeline Start Button
run_from_module_btn = widgets.Button(
    description='[>] Pipeline starten',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px'),
    icon='play',
    tooltip='Startet Pipeline ab gewaehltem Modul'
)

def run_from_module(b):
    """Startet Pipeline ab gewaehltem Modul - MIT INTELLIGENTER CHECKPOINT-VALIDIERUNG"""
    with pipeline_output:
        pipeline_output.clear_output()
        
        start_module = module_dropdown.value
        
        # Re-check Checkpoints
        missing = []
        for module in ALL_MODULES:
            if not (CHECKPOINT_DIR / f"{module}.parquet").exists():
                missing.append(module)
        
        print(f"\n{'='*70}")
        print(f"STARTE PIPELINE AB '{start_module}'")
        print(f"{'='*70}\n")
        
        print("Checkpoint-Status:")
        for i, mod in enumerate(ALL_MODULES, 1):
            status = "[OK]" if mod not in missing else "[--]"
            marker = ">>>" if mod == start_module else "   "
            print(f"{marker} {i:2}. {status} {mod}")
        print()
        
        # INTELLIGENTE VALIDIERUNG: Finde tatsaechliches Startmodul
        actual_start_module = start_module
        
        if start_module != "aggregateData":
            module_index = ALL_MODULES.index(start_module)
            previous_module = ALL_MODULES[module_index - 1]
            previous_checkpoint = CHECKPOINT_DIR / f"{previous_module}.parquet"
            
            if not previous_checkpoint.exists():
                print(f"[!] WARNUNG: Vorheriges Checkpoint '{previous_module}.parquet' fehlt!")
                print(f"[!] CyLinter kann nicht bei '{start_module}' starten ohne vorheriges Checkpoint.")
                print(f"[!] Suche nach letztem existierenden Checkpoint...\n")
                
                # Finde letztes existierendes Checkpoint VOR dem gewaehlten Modul
                found_checkpoint = None
                for i in range(module_index - 1, -1, -1):
                    check_module = ALL_MODULES[i]
                    check_file = CHECKPOINT_DIR / f"{check_module}.parquet"
                    if check_file.exists():
                        found_checkpoint = ALL_MODULES[i + 1]  # Naechstes Modul nach letztem Checkpoint
                        break
                
                if found_checkpoint:
                    actual_start_module = found_checkpoint
                    print(f"[+] Letztes Checkpoint gefunden: {ALL_MODULES[i]}.parquet")
                    print(f"[+] Starte stattdessen bei: {actual_start_module}")
                else:
                    actual_start_module = "aggregateData"
                    print(f"[!] Keine vorherigen Checkpoints gefunden!")
                    print(f"[+] Starte bei: aggregateData (komplette Pipeline)")
                
                print(f"\n[i] Pipeline laeuft von '{actual_start_module}' bis zum Ende")
                print(f"[i] '{start_module}' wird dabei erreicht und neu berechnet.\n")
        
        # BUGFIX: Verwende actual_start_module statt start_module!
        if cylinter_executable.exists():
            cmd = [str(cylinter_executable), "--module", actual_start_module, str(CONFIG_PATH)]
            print(f"[*] Command: cylinter --module {actual_start_module} config.yml")
        else:
            cmd = [str(python_executable), "-m", "cylinter.cylinter", "--module", actual_start_module, str(CONFIG_PATH)]
            print(f"[*] Command: python -m cylinter.cylinter --module {actual_start_module} config.yml")
        
        print(f"\n[!] Pipeline laeuft von '{actual_start_module}' bis zum Ende!")
        print(f"[!] CyLinter ueberschreibt alle Checkpoints ab '{actual_start_module}'!\n")
        
        overall_start = time.time()
        
        try:
            env = os.environ.copy()
            process = subprocess.Popen(
                cmd, cwd=str(BASE_DIR), env=env,
                stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
                universal_newlines=True, bufsize=1
            )
            
            # Stream Output
            for line in process.stdout:
                print(line, end='')
            
            return_code = process.wait()
            total_duration = time.time() - overall_start
            
            # Zusammenfassung
            print(f"\n{'='*70}")
            if return_code == 0:
                print(f"PIPELINE ERFOLGREICH!")
                print(f"   Laufzeit: {total_duration:.1f}s ({total_duration/60:.1f} Min)")
            else:
                print(f"PIPELINE FEHLER (Exit Code: {return_code})")
                print(f"   Laufzeit: {total_duration:.1f}s ({total_duration/60:.1f} Min)")
            print(f"{'='*70}\n")
            
            # Zeige neue Checkpoint-Status
            print("Checkpoint-Status nach Run:")
            for module in ALL_MODULES:
                cp_file = CHECKPOINT_DIR / f"{module}.parquet"
                status = "[OK]" if cp_file.exists() else "[--]"
                print(f"   {status} {module}")
                
        except Exception as e:
            print(f"\n[!] FEHLER: {e}")
            import traceback
            traceback.print_exc()

run_from_module_btn.on_click(run_from_module)

# Tab 2 Layout mit Checkpoint-Info und Scrollbar
checkpoint_info_html = f"""
<div style="background-color: #e7f3ff; padding: 10px; border-left: 4px solid #2196F3; margin-bottom: 10px;">
<b>Checkpoint-Status:</b><br>
Existierend: {len(existing_checkpoints)}/15<br>
Fehlend: {len(missing_checkpoints)}/15<br><br>
<b>Vorgeschlagener Start:</b> <code>{suggested_start}</code><br><br>
<b>Hinweis:</b> CyLinter ueberschreibt alle Checkpoints ab dem Startmodul!
</div>
"""

tab2_title = widgets.HTML("<h3>Checkpoint-Reset & Pipeline-Start</h3>")
tab2_checkpoint_info = widgets.HTML(checkpoint_info_html)
tab2_info = widgets.HTML("""
<div style="background-color: #fff3cd; padding: 10px; border-left: 4px solid #ffc107;">
<b>Funktionen:</b><br>
â€¢ <b>Checkpoint loeschen:</b> Klicken Sie auf Modul-Button oben<br>
â€¢ <b>Pipeline starten:</b> Waehlen Sie Startmodul im Dropdown â†’ "Pipeline starten"<br><br>
<b>WICHTIG:</b><br>
â€¢ Pipeline laeuft vom Startmodul bis zum Ende<br>
â€¢ Vorheriges Checkpoint muss existieren (ausser bei aggregateData)<br>
â€¢ INTELLIGENT: Wenn Checkpoint fehlt, startet bei letztem vorhandenen Checkpoint<br>
â€¢ Alle Checkpoints ab Startmodul werden ueberschrieben
</div>
""")

tab2_content = widgets.VBox([
    tab2_title,
    tab2_checkpoint_info,
    widgets.HTML("<h4>Checkpoint-Reset:</h4>"),
    reset_grid,
    widgets.HTML("<hr><h4>Pipeline starten:</h4>"),
    tab2_info,
    widgets.HBox([module_dropdown, run_from_module_btn]),
    widgets.HTML("<hr><h4>Pipeline-Output:</h4>"),
    pipeline_output
])  # KEINE Hoehenbegrenzung mehr!

# ============================================================================
# TABS KOMBINIEREN MIT SCROLLBAR
# ============================================================================

tabs = widgets.Tab(children=[tab1_content, tab2_content])
tabs.set_title(0, 'Neue Marker (TAG 2+)')
tabs.set_title(1, 'Checkpoint & Pipeline')

main_title = widgets.HTML("<h2>Erweiterte Pipeline-Steuerung</h2>")
main_info = widgets.HTML("""
<div style="background-color: #fff3cd; padding: 15px; border-left: 4px solid #ffc107;">
<b>Diese GUI ist fuer fortgeschrittene Workflows:</b><br><br>
<b>Use Cases:</b><br>
â€¢ <b>TAG 2+:</b> Neue Marker hinzufuegen (Tab 1 â†’ Tab 2)<br>
â€¢ <b>Fehlerkorrektur:</b> Module neu berechnen (Tab 2: Checkpoint loeschen â†’ Pipeline starten)<br>
â€¢ <b>Ab Modul starten:</b> Pipeline ab bestimmtem Punkt fortsetzen (Tab 2)<br><br>
<b>INTELLIGENT:</b> Wenn vorheriges Checkpoint fehlt, startet automatisch bei letztem vorhandenen!<br><br>
<b>Fuer TAG 1 (erste Analyse):</b> Verwenden Sie Zelle 8 + Zelle 9!
</div>
""")

# Main Container - KEINE Hoehenbegrenzung!
ui = widgets.VBox([
    main_title, 
    main_info, 
    tabs
])

# Display
display(ui)

print("\nERWEITERTE PIPELINE-STEUERUNGS-GUI BEREIT!")
print(f"   [*] Tab 1: {len(bio_markers)} Bio-Marker verfuegbar")
print(f"   [*] Tab 2: {len(ALL_MODULES)} Pipeline-Module")
print(f"   [*] Tab 2: Checkpoints: {len(existing_checkpoints)} OK, {len(missing_checkpoints)} fehlend")
print(f"   [*] Tab 2: Vorgeschlagener Start: {suggested_start}")
print(f"   [*] Tab 2: INTELLIGENTE Validierung aktiviert!\n")


## ?? Parquet Viewer (optional)

Optionales Werkzeug, um Checkpoint-Parquet-Dateien einzusehen. L?dt keine Pipeline; vorher Zelle 11 ausf?hren, damit `CHECKPOINT_DIR` gesetzt ist.


In [ ]:
# PARQUET DATA VIEWER - Kopieren Sie diesen gesamten Code in eine neue Notebook-Zelle

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, HTML
from pathlib import Path
import os

checkpoint_dir = CHECKPOINT_DIR
parquet_files = sorted(checkpoint_dir.glob('*.parquet'))
parquet_names = [f.name for f in parquet_files]

if not parquet_files:
    print("âŒ No parquet files found!")
else:
    print(f"âœ… Found {len(parquet_files)} parquet file(s)")

current_data = None

title_html = widgets.HTML(value="""<div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px;'><h2 style='color: white; margin: 0;'>ðŸ“Š Parquet Viewer</h2></div>""")

file_dropdown = widgets.Dropdown(options=parquet_names, value=parquet_names[-1] if parquet_names else None, description='File:', layout=widgets.Layout(width='500px'))
load_btn = widgets.Button(description='ðŸ“‚ Load', button_style='primary')
info_output = widgets.Output()
preview_output = widgets.Output(layout=widgets.Layout(max_height='400px', overflow_y='auto'))
export_filename = widgets.Text(value='data.csv', description='Name:')
export_btn = widgets.Button(description='ðŸ’¾ Export CSV', button_style='success')

def load_data(b):
    global current_data
    with info_output:
        info_output.clear_output()
        try:
            current_data = pd.read_parquet(checkpoint_dir / file_dropdown.value)
            print(f"âœ… Loaded {len(current_data):,} rows, {len(current_data.columns)} cols")
            with preview_output:
                preview_output.clear_output()
                display(current_data.head(10))
        except Exception as e:
            print(f"âŒ Error: {e}")

def export_data(b):
    with info_output:
        info_output.clear_output()
        if current_data is not None:
            path = Path.cwd() / export_filename.value
            current_data.to_csv(path, index=False)
            print(f"âœ… Exported to {path}")
        else:
            print("âŒ Load data first!")

load_btn.on_click(load_data)
export_btn.on_click(export_data)

ui = widgets.VBox([title_html, widgets.HBox([file_dropdown, load_btn]), info_output, preview_output, widgets.HBox([export_filename, export_btn])], layout=widgets.Layout(padding='20px'))
display(ui)